# Problem 2

## (a) 
### Hidden layer:

1D Input → use shallow network (1–2 hidden layers)

2D or more dimension input → may need deeper network

### Node number:

Smooth, simple function→ small width (16–32)

Oscillatory / curved function→ moderate width (32–64)

Very complex function → larger width or extra layer

For this question, I choose 1 hidden layer and 32 nodes

## (b)
### Hidden layers: 

use ReLU as a good first choice (remember, it's just a good default, not a mandatory one).

### Output layer:

Regression (predict a real number, e.g., stock price differences): use Linear output.

Regression (predict a non-negative number, e.g., heights): use ReLU output. A special case of this is - predicting values between 0 and 1, say Probabilities - use Sigmoid output.

Binary classification (yes/no): use Sigmoid output (one node).

Multi-class classification (one of many classes): use Softmax (not covered yet in class) - don’t use a single sigmoid for this.

### My answer:
Hidden layer: ReLU

output layer: Linear

## (c)



![Loss Function](loss_function.png)

based on this rule, I choose MSE

## (d)

In [1]:
import os

In [2]:
print(os.listdir())

['.git', '.vscode', 'colleges-sample.csv', 'data.csv', 'dsc140b-homework06.ipynb', 'graphfolder', 'HW05Q02.ipynb', 'HW05Q03.ipynb', 'HW07.ipynb', 'hw7_data.csv', 'loss_function.png', 'README.md', 'requirements.txt']


In [4]:
# import data
!pip install pandas
import pandas as pd
df = pd.read_csv("hw7_data.csv")
print(df)

   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------- ----- 8.4/9.7 MB 54.5 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 48.5 MB/s  0:00:00
   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ---------------------------------------- 12.3/12.3 MB 93.8 MB/s  0:00:00

   ---------------------------------------- 0/3 [tzdata]
   ---------------------------------------- 0/3 [tzdata]
   ---------------------------------------- 0/3 [tzdata]
   ---------------------------------------- 0/3 [tzdata]
   ---------------------------------------- 0/3 [tzdata]
   ---------------------------------------- 0/3 [tzdata]
   ---------------------------------------- 0/3 [tzdata]
   ---------------------------------------- 0/3 [tzdata]
   ------------- -------------------------- 1/3 [numpy]
   ------------- -------------------------- 1/3 [numpy]
   ------------- -------------------------- 1/3 [numpy]
   ------


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


    3.74540  -0.54167
0   9.50714  -0.17197
1   7.31994   0.88828
2   5.98658  -0.88854
3   1.56019   0.93404
4   1.55995   1.10707
..      ...       ...
94  4.93796  -1.21941
95  5.22733  -0.89345
96  4.27541  -0.80369
97  0.25419   0.33447
98  1.07891   1.12960

[99 rows x 2 columns]


In [24]:
# UPDATEEEEEEEEEEEEEEEEEE